### NOTE : The model you'll generate using this code and base data (foodcsv.xlsx) is already in this repo in folder FoodNER. You can use that directly in FoodItemRating Notebook. 

You can use this notebook if you have extracted more labelled data for custom NER to make the custom NER model better

In [3]:
from __future__ import unicode_literals,print_function
import pandas as pd
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm

class TrainFoodNER():
    
    def __init__(self, df_path):
        self.df_path = df_path
        
    def generate_train_from_df(self):
        """This function generates the Train data in the format accepted by spacy from our csv file for training
           custom NER model and returns the training data variable created thereafter"""
        df = pd.read_excel(self.df_path)
        train = []
        gone = set()
        for index, value in df.iterrows():
            if index in gone:
                continue
            group = tuple()
            di = dict()
            li = []
            data = tuple()
            if pd.notnull(value['Start']) and pd.notnull(value['End']):
                data = data + (int(value['Start']),int(value['End']),value['Tag'].upper())
                li.append(data)
            c = index
            if ((c+1==df.shape[0])):
                break
            while(df.loc[c+1,'Review']==df.loc[index,'Review'] ):
                gone.add(c+1)
                data = tuple()
                if pd.notnull(df.loc[c+1,'Start']) and pd.notnull(df.loc[c+1,'End']):
                    data = data + (int(df.loc[c+1,'Start']),int(df.loc[c+1,'End']),df.loc[c+1,'Tag'].upper())
                    li.append(data)
                c = c+1
                if ((c+1==df.shape[0])):
                    break
            di['entities'] = li
            group = group + (value['Review'],di)
            train.append(group)
        return train

    def train_ner(self, output_dr):
        """This function trains the ner model and saves the model in directory provided as parameter to function"""
        """Parameters:
           output_dr -> Path where you want the model to be saved, must be string (Ex. '/users/xyz/desktop')
           """
        model = None
        output_dir=Path(output_dr)
        n_iter=100
        if model is not None:
            nlp = spacy.load(model)  # load existing spaCy model
            print("Loaded model '%s'" % model)
        else:
            nlp = spacy.blank('en')  # create blank Language class
            print("Created blank 'en' model")

        if 'ner' not in nlp.pipe_names:
            ner = nlp.create_pipe('ner')
            nlp.add_pipe(ner, last=True)
        # otherwise, get it so we can add labels
        else:
            ner = nlp.get_pipe('ner')

        # add labels
        TRAIN_DATA = self.generate_train_from_df()
        for _, annotations in TRAIN_DATA:
            for ent in annotations.get('entities'):
                ner.add_label(ent[2])

        other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
        with nlp.disable_pipes(*other_pipes):  # only train NER
            optimizer = nlp.begin_training()
            for itn in range(n_iter):
                random.shuffle(TRAIN_DATA)
                losses = {}
                print(itn)
                for text, annotations in tqdm(TRAIN_DATA):
                    nlp.update(
                        [text],  # batch of texts
                        [annotations],  # batch of annotations
                        drop=0.35,  # dropout - make it harder to memorise data
                        sgd=optimizer,  # callable to update weights
                        losses=losses)
                print(losses)
                if n_iter%10==0:
                    if output_dir is not None:
                        output_dir = Path(output_dir)
                        if not output_dir.exists():
                            output_dir.mkdir()
                        nlp.to_disk(output_dir)
                        print("Saved model to", output_dir)

        if output_dir is not None:
            output_dir = Path(output_dir)
            if not output_dir.exists():
                output_dir.mkdir()
            nlp.to_disk(output_dir)
            print("Saved model to", output_dir)
            
def generate_food_model(df_path,output_dir):
    """This function is the only one needed to be called to be called to generate FOOD NER model"""
    """Parameters:
       df_path : Path of excel file containing training data
       output_dir : Directory where you want the model to be saved (must be string, example : '/Users/XYZ/desktop')"""
    obj = TrainFoodNER(df_path = df_path)
    obj.train_ner(output_dir)

In [ ]:
# Main and kinda only function to be called
# Path of data for model to be trained from (must be excel, else change in above class function if uploading csv),
# Path where you want to store the custom Model
generate_food_model('/Users/rishabh/Desktop/ProjectZomato/foodcsv.xlsx','/Users/rishabh/Desktop/ProjectZomato/FoodNER')

Created blank 'en' model


  0%|          | 0/316 [00:00<?, ?it/s]/Applications/anaconda3/lib/python3.6/site-packages/numpy/linalg/linalg.py:2257: RuntimeWarning: invalid value encountered in sqrt
  ret = sqrt(sqnorm)
  0%|          | 1/316 [00:00<00:55,  5.68it/s]

0


  0%|          | 0/316 [00:00<?, ?it/s]

{'ner': 949.6090647514052}
Saved model to /Users/rishabh/Desktop/ProjectZomato
1


...

{'ner': 23.575507888034096}
Saved model to /Users/rishabh/Desktop/ProjectZomato
98


  0%|          | 0/316 [00:00<?, ?it/s]

{'ner': 11.026393699172496}
Saved model to /Users/rishabh/Desktop/ProjectZomato
99


100%|██████████| 316/316 [01:26<00:00,  3.67it/s]


{'ner': 27.150179142321708}
Saved model to /Users/rishabh/Desktop/ProjectZomato
Saved model to /Users/rishabh/Desktop/ProjectZomato


# Custom NER performance

Since I only have 316 examples, I haven't split the dataset before into train and validation. So the metric scores we'll be getting will be on train data below. I have also retrained the model again dividing data into train and validation (files foodcsvtrain.xlsx and foodcsvtest.xlsx) which will be right after train precision calculated score

I will be using **Precision Score** as the main metric for model evaluation. 

This is so because I am more interested to find out how many of the named entities is my model mis-classifying as being a food item. Anything classified multiple times as a food item with high rating will be shown to any end user and obviously you don't want it to output things like 'CP', 'Kirti Nagar Momos', 'Barbeque Nation'. If intuition of using precision is still not clear enough to you, take an example : A food item which is not being recognised as a food item will not affect our output that much since if it's trending or that good, there will be a lot of reviews having that and chances of our NER model recognising it increases. However a word or a Group of words recognised as food item which essentially are not will look wrongfully put in the output.

In [4]:
def get_precision(model_path,train_path):
    """The function takes in model path of Custom Food NER and file path of data it was trained from and returns
    the precision score of the model on train data"""
    nlp = spacy.load(model_path)
    tp=0
    fp=0
    obj = TrainFoodNER('/Users/rishabh/Desktop/ProjectZomato/foodcsv.xlsx')
    TRAIN_DATA = obj.generate_train_from_df()
    for items in TRAIN_DATA:
        doc = nlp(items[0])
        k_list = []
        for its in items[1]['entities']:
            k_list.append(items[0][its[0]:its[1]])
        for tokens in doc.ents:
            if tokens.text in k_list:
                tp+=1
            else:
                fp+=1
    return tp/(tp+fp)

In [36]:
get_precision('/Users/rishabh/Desktop/ProjectZomato/FoodNER','/Users/rishabh/Desktop/ProjectZomato/foodcsv.xlsx')

0.9097222222222222

A precision of 0.90 is good but two things are to be kept in mind, it's on train data, it can be so that the model is overfitting. To find this out I can divide 316 examples I trained on above into train and validation, construct model on train and evaluate on validation which I have done below.

## Let's try to train it on 245 examples and validate on remaining 66 examples:

In [ ]:
generate_food_model('/Users/rishabh/Desktop/ProjectZomato/foodcsvtrain.xlsx','/Users/rishabh/Desktop/ProjectZomato/trainNER')

Created blank 'en' model


  0%|          | 0/245 [00:00<?, ?it/s]/Applications/anaconda3/lib/python3.6/site-packages/numpy/linalg/linalg.py:2257: RuntimeWarning: invalid value encountered in sqrt
  ret = sqrt(sqnorm)
  0%|          | 1/245 [00:00<00:42,  5.74it/s]

0


  0%|          | 0/245 [00:00<?, ?it/s]

{'ner': 754.4667709981615}
Saved model to /Users/rishabh/Desktop/ProjectZomato/trainNER
1


...

{'ner': 25.161231756501675}
Saved model to /Users/rishabh/Desktop/ProjectZomato/trainNER
98


  0%|          | 0/245 [00:00<?, ?it/s]

{'ner': 15.362498195109108}
Saved model to /Users/rishabh/Desktop/ProjectZomato/trainNER
99


100%|██████████| 245/245 [01:07<00:00,  3.64it/s]


{'ner': 17.557805938945133}
Saved model to /Users/rishabh/Desktop/ProjectZomato/trainNER
Saved model to /Users/rishabh/Desktop/ProjectZomato/trainNER


In [5]:
get_precision('/Users/rishabh/Desktop/ProjectZomato/trainNER','/Users/rishabh/Desktop/ProjectZomato/foodcsvtest.xlsx')

0.8337236533957846

A precision of 83% on validation is decent enough I would say, spacy did a fantastic job with just 245 samples for training. Although keep in mind These samples should be increased to make model better and more robust. It might have been a sampling skew from my manual data collection that I only picked training examples of certain kind. And obviously 245 examples cannot be close to representing all kinds of reviews on zomato. But nevertheless it's still better. I will be using model trained on entire data and not just train and validation data.